In [1]:
%load_ext autoreload
%autoreload 2

In [15]:
import sys
sys.path.append('../src/')
from plotutils import plotts
from wahooreader import WahooTickrX
from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot, Column, Row
from bokeh.models.widgets import DatePicker
from bokeh.models import CustomJS, ColumnDataSource, HoverTool, Legend
from bokeh.plotting import figure

from datetime import datetime
import glob
import os
import time
from pathlib import Path
import pandas as pd
import dask
import dask.dataframe as dd
from dask.diagnostics import ProgressBar
import datetime

output_notebook()

Loading BokehJS ...

In [123]:
!python ../src/fitetl.py

[########################################] | 100% Completed | 22.4s


In [124]:
start = datetime.datetime.today() - pd.Timedelta('28 days')

# HR Recovery Trends

In [127]:
datadir_hrsum = '/Users/hasannagib/Documents/s3stage/wahoo/heartrate_sumstat/'
df = dd.read_csv(Path(f'{datadir_hrsum}*.csv')).compute()
df = df.rename(columns={'Unnamed: 0': 'timestamp'})
df['timestamp'] = pd.to_datetime(df['timestamp'])
df = df.set_index('timestamp')

df_bar = df.copy() 
df_bar['L0'] = 22
df_bar['L1'] = 53
df_bar['L2'] = 59
df_bar['L3'] = 66

p1 = plotts(
    df_bar[['L0','L1', 'L2','L3','120_sec_rec']].loc[start:], 
    units=['bpm'],
    styles=['--']*4 + 2*['|'], 
    title='120 sec HR recovery trend',
    plot_width=450
);

# HR Zone Trends

In [129]:
p2 = plotts(
    (df.rolling(7).sum().dropna()/60).loc[start:], 
    ys=['174_', '152_173', '138_151'],
    styles=['o-'],
    units=['min'],
    title='Time spent in HR zones (7 day rolling sum)',
    ylabel='Minutes',
    plot_width=450,
);

# HR Time Series

In [149]:
ts_files = sorted(glob.glob('/Users/hasannagib/Documents/s3stage/wahoo/heartrate_ts/*.csv'))

@dask.delayed
def read_ts(file):
    df = pd.read_csv(file,parse_dates=['timestamp']
    ).set_index('timestamp').sort_index().reset_index()[
        ['heart_rate']
    ].rename(columns={
        'heart_rate':pd.to_datetime(os.path.basename(file)[:-11]).strftime('%a %b %d %Y'),
        
    })
    return df

dfs = [read_ts(file) for file in ts_files]

with ProgressBar():
    dfs = dask.compute(dfs)[0]

df = pd.concat(dfs, axis=1).reset_index().rename(columns={'index':'s'})
df['Time'] = df['s'].apply(lambda x: time.strftime('%H:%M:%S', time.gmtime(x)))
df['BPM'] = df.iloc[:,-2]

[########################################] | 100% Completed |  0.6s


In [164]:
p3 = figure(
    width=900, 
    height=350, 
    title='Heart Rate',
    x_axis_label='Time (seconds)',
    y_axis_label='BPM',
    toolbar_location="above",
    tooltips=[
        ('Time','@Time'),
        ('BPM', '@BPM'),
    ]
)

cds = ColumnDataSource(df)
p3.line('s', 'BPM', source=cds, color="#993333")

callback = CustomJS(
    args={'source':cds},
    code=
    """
    console.log('changed selected option', cb_obj.value);
    var src_data = source.data;
    var yval = cb_obj.value;
    src_data['BPM'] = src_data[yval];
    source.change.emit();
    """
)

datePicker = DatePicker(title='Workout date:', width=200, value=df.columns[-3])
datePicker.js_on_change('value', callback)

# Dashboard

In [165]:
show(Column(Row(p1, p2), datePicker, p3))